In [ ]:
import cv2, sys
import tensorflow as tf

In [ ]:
print("Python exe:", sys.executable)
print("cv2 version:", cv2.__version__)
print("cv2 file   :", cv2.__file__)

In [ ]:
cv2.cuda.getCudaEnabledDeviceCount()

In [ ]:
info = cv2.getBuildInformation()
print("\n--- Resumen CUDA/Python ---")
for line in info.splitlines():
    if "NVIDIA CUDA:" in line or "cuDNN:" in line or "NVIDIA GPU arch:" in line or line.strip().startswith("Python 3"):
        print(line)

In [ ]:
print("\nCUDA devices:", cv2.cuda.getCudaEnabledDeviceCount())
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    for i in range(cv2.cuda.getCudaEnabledDeviceCount()):
        device_info = cv2.cuda.printCudaDeviceInfo(i)
        print(device_info)
        #print(f" Device {i}:", cv2.cuda.DeviceInfo(i).name())

In [ ]:
import io
import contextlib

def get_device_name(index: int) -> str:
    """
    Devuelve el nombre del dispositivo CUDA usando printShortCudaDeviceInfo.
    """
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        cv2.cuda.printShortCudaDeviceInfo(index)
    salida = buf.getvalue().splitlines()
    if not salida:
        return f"Device {index}"
    # Normalmente la primera línea es tipo: "Device 0: NVIDIA GeForce RTX 4060"
    primera = salida[0].strip()
    if ":" in primera:
        return primera.split(":", 1)[1].strip()
    return primera

def listar_dispositivos_cuda_detallado():
    if not hasattr(cv2, "cuda"):
        print("Esta versión de OpenCV no tiene el módulo cv2.cuda.")
        return

    try:
        num_devices = cv2.cuda.getCudaEnabledDeviceCount()
    except Exception as e:
        print("Error obteniendo el número de dispositivos CUDA:", e)
        return

    if num_devices == 0:
        print("No se detectan GPUs CUDA o OpenCV no fue compilado con soporte CUDA.")
        return

    print(f"GPUs CUDA detectadas: {num_devices}\n")

    for i in range(num_devices):
        nombre = get_device_name(i)
        print("=" * 70)
        print(f"Dispositivo #{i} - {nombre}")
        print("=" * 70)
        cv2.cuda.printCudaDeviceInfo(i)
        print()

In [ ]:
listar_dispositivos_cuda_detallado()

In [ ]:
print("TensorFlow version:", tf.__version__)

# 1. Listar dispositivos físicos
physical_devices = tf.config.list_physical_devices()
print("\nDispositivos físicos visibles:")
for d in physical_devices:
    print("  -", d)

gpus = tf.config.list_physical_devices("GPU")
print("\nGPUs detectadas por TensorFlow:", gpus)

if not gpus:
    print("\n❌ TensorFlow NO ve GPUs. Está corriendo solo en CPU.")
else:
    print("\n✅ TensorFlow ve al menos una GPU. Probando una operación en la GPU...")

    # 2. Probar operación sencilla en la GPU
    try:
        with tf.device("/GPU:0"):
            a = tf.random.normal((1000, 1000))
            b = tf.random.normal((1000, 1000))
            c = tf.matmul(a, b)

        print("Resultado de matmul:", c)
        print("Dispositivo donde se ejecutó c:", c.device)

        if "GPU" in c.device.upper():
            print("\n🎉 Todo bien: la operación se ejecutó en la GPU.")
        else:
            print("\n⚠ La GPU existe, pero esta operación parece ejecutarse en CPU.")
    except Exception as e:
        print("\n⚠ Ocurrió un error al intentar usar la GPU:")
        print(e)

In [ ]:
from ultralytics import YOLO
import torch